In [ ]:
from flask import Flask, request, jsonify, send_file
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow_datasets as tfds

# Load model and dataset
model_path = 'model/caltech101_cnn_model.keras'
model = tf.keras.models.load_model(model_path)

# Load dataset
tfds_dir = "data/"
dataset, info = tfds.load("caltech101", as_supervised=True, with_info=True, data_dir=tfds_dir)
test_data = dataset["test"].map(lambda img, lbl: (tf.image.resize(img, (128, 128)) / 255.0, lbl)).batch(32)

# Flask App
app = Flask(__name__)

# Prediction Function
def visualize_predictions(dataset, model, num_images=16, output_file="output/Visualize_Prediction.png"):
    class_names = info.features['label'].names
    for images, labels in dataset.take(1):
        preds = model.predict(images)
        pred_labels = np.argmax(preds, axis=1)

        plt.figure(figsize=(8, 8))
        for i in range(min(num_images, images.shape[0])):
            plt.subplot(4, 4, i + 1)
            plt.imshow(images[i].numpy())
            plt.title(f"True: {class_names[labels[i]]}\nPred: {class_names[pred_labels[i]]}")
            plt.axis('off')
        plt.tight_layout()
        plt.savefig(output_file)
        plt.close()

@app.route('/visualize_predictions', methods=['GET'])
def handle_visualize_predictions():
    try:
        num_images = int(request.args.get('num_images', 16))
        output_file = "output/Visualize_Prediction.png"
        
        visualize_predictions(test_data, model, num_images, output_file)
        
        return send_file(output_file, mimetype='image/png')
    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    os.makedirs('outputs', exist_ok=True)
    app.run(host='0.0.0.0', port=5000)


2024-12-12 17:15:49.445030: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-12 17:15:49.464518: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-12 17:15:49.493681: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-12 17:15:49.505979: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733998549.516159  107639 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733998549.51

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://172.29.238.121:5001
INFO:werkzeug:Press CTRL+C to quit
2024-12-12 17:16:02.862465: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:376] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608
2024-12-12 17:16:02.902423: W tensorflow/core/kernels/data/cache_dataset_ops.cc:914] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 627ms/step


2024-12-12 17:16:03.889242: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
INFO:werkzeug:127.0.0.1 - - [12/Dec/2024 17:16:03] "GET /visualize_predictions?num_images=12 HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step


2024-12-12 17:16:47.133054: W tensorflow/core/kernels/data/cache_dataset_ops.cc:914] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2024-12-12 17:16:47.644136: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
INFO:werkzeug:127.0.0.1 - - [12/Dec/2024 17:16:47] "GET /visualize_predictions?num_images=15 HTTP/1.1" 200 -
